# 1. Ask an interesting question

1. Delay binary classifier (yes/no, >15 min)
- Better isolate the key features to predict delays so that airlines focus their efforts on their weakest points
- Account for uncontrollable variables (e.g. unpredicted adverse weather) and prepare to act consequently
2. Additional questions:
- When is the best time of day/day of week/time of year to fly to minimize delays?
- Do older planes suffer more delays?
- How does the number of people flying between different locations change over time?
- How well does weather predict plane delays?
- Can you detect cascading failures as delays in one airport create delays in others? Are there critical links in the system?

___

# 2. Get the data

### On-Time : Reporting Carrier On-Time Performance (1987-present)

Source: https://www.transtats.bts.gov/DatabaseInfo.asp?DB_ID=120&DB_URL=Mode_ID=1&Mode_Desc=Aviation&Subject_ID2=0

<em>Note</em>: Over time both the code and the name of a carrier may change and the same code or name may be assumed by a different airline. To ensure that you are analyzing data from the same airline, TranStats provides four airline-specific variables that identify one and only one carrier or its entity: Airline ID (AirlineID), Unique Carrier Code (UniqueCarrier), Unique Carrier Name (UniqueCarrierName), and Unique Entity (UniqCarrierEntity). A unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation. US Airways and America West started to report combined on-time data in January 2006 and combined traffic and financial data in October 2007 following their 2005 merger announcement. Delta and Northwest began reporting jointly in January 2010 following their 2008 merger announcement. Continental Micronesia was combined into Continental Airlines in December 2010 and joint reporting began in January 2011. Atlantic Southeast and ExpressJet began reporting jointly in January 2012. United and Continental began reporting jointly in January 2012 following their 2010 merger announcement. Endeavor (9E) operated as Pinnacle prior to August 2013. Envoy (MQ) operated as American Eagle prior to April 2014. Southwest (WN) and AirTran (FL) began reporting jointly in January 2015 following their 2011 merger announcement. American (AA) and US Airways (US) began reporting jointly as AA in July 2015 following their 2013 merger announcement. Alaska (AS) and Virgin America (VX) began reporting jointly as AS in April 2018 following their 2016 merger announcement.

        
- **AC_Types**:
    - **Summary**:
        - Reporting Carrier On-Time Performance (1987-present)
    - **Description**:
        - Reporting carriers are required to (or voluntarily) report on-time data for flights they operate: on-time arrival and departure data for non-stop domestic flights by month and year, by carrier and by origin and destination airport. Includes scheduled and actual departure and arrival times, canceled and diverted flights, taxi-out and taxi-in times, causes of delay and cancellation, air time, and non-stop distance. Use Download for individual flight data.
    - **File**:
        - YYMM_123456789_T_ONTIME_REPORTING.zip *(YY = Year ; MM = Month)*

In [1]:
# Import libraries to be used

# Warning messages display
## import warnings
## warnings.filterwarnings(action='ignore') # https://docs.python.org/3/library/warnings.html#the-warnings-filter

# Directories/Files management
import os.path
## from zipfile import ZipFile # De momento no ha hecho falta 

# Timing
import time

# Memory monitoring
%load_ext memory_profiler
### Use '%memit' to check at each point

# Data analysis and wrangling
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # Show all columns in DataFrames
## pd.set_option('display.max_rows', None) # It greatly slows down the output display and freezes the kernel

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot') # choose a style: 'plt.style.available'
sns.set_theme(context='notebook',
              style="darkgrid") # {darkgrid, whitegrid, dark, white, ticks}
palette = sns.color_palette("flare", as_cmap=True);
import altair as alt

# Machine Learning
## from sklearn.[...] import ...

In [2]:
t0 = time.perf_counter() 

In [3]:
# Detect Operating System running and manage paths accordingly

if os.name == 'nt': # Windows
    root = r"C:\Users\turge\CompartidoVM\0.TFM"
    print("Running on Windows.")
elif os.name == 'posix': # Ubuntu
    root = "/home/dsc/shared/0.TFM"
    print("Running on Ubuntu.")
print("root path\t", root)

Running on Windows.
root path	 C:\Users\turge\CompartidoVM\0.TFM


___

Additional information on each column meaning can be found [here](https://www.transtats.bts.gov/Fields.asp?Table_ID=236&SYS_Table_Name=T_ONTIME_REPORTING&User_Table_Name=Reporting%20Carrier%20On-Time%20Performance%20(1987-present)&Year_Info=1&First_Year=1987&Last_Year=2020&Rate_Info=0&Frequency=Monthly&Data_Frequency=Annual,Quarterly,Monthly).

___

### SAMPLE FILE

Single file import (i.e. month-sized database)

Considering that each file is considerably big, and that later on many of them will have to be grouped, a first exploration will be done considering the first 10,000 rows.

Let's see which columns will remain, and skip the rest.

___

### MULTIPLE FILE

Let's proceed with multiple-file importing, through concatenation into a single DataFrame.

___

#### Run only the first time to read the 12 month individual files and generate the global file (entire year 2019)

##### 1. Read individual month files

##### 2. Generate file (2019)

___

In [4]:
%memit

peak memory: 135.92 MiB, increment: 0.24 MiB


In [5]:
csv_path = os.path.join(root,
                        "Output_Data",
                        "US_DoT",
                        "AL_OTP_MVP_Preprocessed_19_v1.csv")
csv_path

'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\US_DoT\\AL_OTP_MVP_Preprocessed_19_v1.csv'

In [6]:
cols = [
    
### -----  < X > (PRE-FLIGHT DATA) -----

# Time Period
#  'YEAR', # Disregarded: for the time being, analysis limted to 2019
#  'QUARTER', # Disregarded: redundant
 'MONTH',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
#  'FL_DATE', # Disregarded: redundant
# Airline / Aircraft
 'OP_UNIQUE_CARRIER',
#  'OP_CARRIER_AIRLINE_ID', # Disregarded: redundant
#  'OP_CARRIER', # Disregarded: redundant
 'TAIL_NUM',
#  'OP_CARRIER_FL_NUM', # Unknown in advance?
# Origin
#  'ORIGIN_AIRPORT_ID', # Disregarded: redundant
#  'ORIGIN_AIRPORT_SEQ_ID', # Disregarded: redundant
#  'ORIGIN_CITY_MARKET_ID', # Disregarded: not relevant for this particular analysis
 'ORIGIN',
#  'ORIGIN_CITY_NAME', # Disregarded: redundant
#  'ORIGIN_STATE_ABR', # Disregarded: redundant
#  'ORIGIN_STATE_FIPS', # Federal Information Processing Standards # Not used for the moment
#  'ORIGIN_STATE_NM', # Disregarded: redundant
#  'ORIGIN_WAC', # World Area Code # Not used for the moment
# Destination
#  'DEST_AIRPORT_ID', # Disregarded: redundant
#  'DEST_AIRPORT_SEQ_ID', # Disregarded: redundant
#  'DEST_CITY_MARKET_ID', # Disregarded: not relevant for this particular analysis
 'DEST',
#  'DEST_CITY_NAME', # Disregarded: redundant
#  'DEST_STATE_ABR', # Disregarded: redundant
#  'DEST_STATE_FIPS', # Federal Information Processing Standards # Not used for the moment
#  'DEST_STATE_NM', # Disregarded: redundant
#  'DEST_WAC', # World Area Code # Not used for the moment
# Departure Performance
 'CRS_DEP_TIME',
 'DEP_TIME_BLK', # Will be used for grouping into categories
#  'TAXI_OUT_median', #  Output / However, the median for each airport could be used as input !! (explanation below)   
# Arrival Performance
 'CRS_ARR_TIME',
 'ARR_TIME_BLK', # Will be used for grouping into categories
#  'TAXI_IN_median', #  Output / However, the median for each airport could be used as input !! (explanation below) 
# Flight Summaries
 'CRS_ELAPSED_TIME',
 'FLIGHTS',
 'DISTANCE',
 'DISTANCE_GROUP',

### ----- < y > (PRE-FLIGHT DATA) -----

# Departure Performance
 'DEP_TIME', # Disregarded: redundant
 'DEP_DELAY', # Disregarded: other potentially useful target
#  'DEP_DELAY_NEW', # Disregarded: redundant
 'DEP_DEL15', # Disregarded: other potentially useful target
#  'DEP_DELAY_GROUP', # Disregarded: not relevant for this particular analysis
 'TAXI_OUT', #  Output / However, the median for each airport could be used as input !! (explanation below)
#  'WHEELS_OFF', # Disregarded: redundant
# Arrival Performance
#  'WHEELS_ON', # Disregarded: redundant
 'TAXI_IN', #  Output / However, the median for each airport could be used as input !! (explanation below)
 'ARR_TIME', # Disregarded: redundant
 'ARR_DELAY', # -------------------------------------------> MAIN TARGET !! (i.e. < y >)
#  'ARR_DELAY_NEW', # Disregarded: redundant
 'ARR_DEL15', # Disregarded: other potentially useful target
#  'ARR_DELAY_GROUP', # Disregarded: not relevant for this particular analysis
# Cancellations and Diversions
 'CANCELLED', # Disregarded: not relevant for this particular analysis
#  'CANCELLATION_CODE', # Disregarded: not relevant for this particular analysis
#  'DIVERTED', # Disregarded: not relevant for this particular analysis
# Flight Summaries
#  'ACTUAL_ELAPSED_TIME', # Disregarded: redundant
#  'AIR_TIME', # Disregarded: redundant
# Cause of Delay
 'CARRIER_DELAY', # Disregarded: other potentially useful target
 'WEATHER_DELAY', # Disregarded: other potentially useful target
 'NAS_DELAY', # Disregarded: other potentially useful target
 'SECURITY_DELAY', # Disregarded: other potentially useful target
 'LATE_AIRCRAFT_DELAY', # Disregarded: other potentially useful target
# Gate Return Information at Origin Airport (Data starts 10/2008)
#  'FIRST_DEP_TIME', # Disregarded: not relevant for this particular analysis
#  'TOTAL_ADD_GTIME', # Disregarded: not relevant for this particular analysis
#  'LONGEST_ADD_GTIME', # Disregarded: not relevant for this particular analysis
# Diverted Airport Information (Data starts 10/2008)
#  'DIV_AIRPORT_LANDINGS', # Disregarded: not relevant for this particular analysis
]

In [7]:
%%time

# Chunk-loading procedure is required so as to prevent memory-saturation errors from appearing:

# cols = pd.read_csv(csv_path, nrows=1).columns

chunks_list = []
chunks = pd.read_csv(csv_path,
                     encoding='latin1',
#                      nrows=1e6, # Fail-safe: in case the file is inadvertently too big
                     chunksize=1e6,
                     usecols=cols[:],
                     low_memory = False)

for i, chunk in enumerate(chunks):
    if i == 13: # Fail-safe: for debugging purposes only
        break
    chunks_list.append(chunk)

    
df = pd.concat(chunks_list, axis=0)
del chunks_list

df.sample(5)

Wall time: 1min 13s


,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,DEP_TIME_BLK,TAXI_OUT,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,ARR_TIME_BLK,CANCELLED,CRS_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
5503238,9,2,1,AA,N753US,PHL,RSW,1352,1343.0,-9.0,0.0,1300-1359,17.0,4.0,1636,1618.0,-18.0,0.0,1600-1659,0.0,164.0,1.0,993.0,4,NaN,NaN,NaN,NaN,NaN
2618832,5,16,4,WN,N8640D,DEN,LAS,1220,1222.0,2.0,0.0,1200-1259,11.0,4.0,1315,1308.0,-7.0,0.0,1300-1359,0.0,115.0,1.0,628.0,3,NaN,NaN,NaN,NaN,NaN
6828195,12,1,7,YV,N245LR,PHX,TUS,1112,1110.0,-2.0,0.0,1100-1159,27.0,4.0,1207,1203.0,-4.0,0.0,1200-1259,0.0,55.0,1.0,110.0,1,NaN,NaN,NaN,NaN,NaN
5418686,9,15,7,WN,N747SA,MCI,DAL,1130,1132.0,2.0,0.0,1100-1159,8.0,12.0,1305,1259.0,-6.0,0.0,1300-1359,0.0,95.0,1.0,461.0,2,NaN,NaN,NaN,NaN,NaN
682151,2,3,7,UA,N27246,MCO,CLE,1038,1031.0,-7.0,0.0,1000-1059,18.0,4.0,1303,1251.0,-12.0,0.0,1300-1359,0.0,145.0,1.0,895.0,4,NaN,NaN,NaN,NaN,NaN


In [8]:
# For On-Time flights, the "..._DELAY" columns show NaN.
# Actually each of them could be any number provided that the sum of them is less than 15 minutes.
# However, for data consistency, let's replace them by 0:
causes = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
df[causes] = df[causes].fillna(value=0)
df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,DEP_TIME_BLK,TAXI_OUT,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,ARR_TIME_BLK,CANCELLED,CRS_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,3,4,9E,N195PQ,TYS,ATL,1140,1205.0,25.0,1.0,1100-1159,30.0,4.0,1250,1315.0,25.0,1.0,1200-1259,0.0,70.0,1.0,152.0,1,0.0,0.0,0.0,0.0,25.0
1,1,4,5,9E,N919XJ,TYS,ATL,1140,1250.0,70.0,1.0,1100-1159,35.0,9.0,1250,1412.0,82.0,1.0,1200-1259,0.0,70.0,1.0,152.0,1,0.0,0.0,12.0,0.0,70.0
2,1,5,6,9E,N316PQ,ATL,SGF,950,956.0,6.0,0.0,0900-0959,20.0,3.0,1051,1043.0,-8.0,0.0,1000-1059,0.0,121.0,1.0,563.0,3,0.0,0.0,0.0,0.0,0.0
3,1,6,7,9E,N325PQ,ATL,SGF,950,945.0,-5.0,0.0,0900-0959,16.0,3.0,1053,1029.0,-24.0,0.0,1000-1059,0.0,123.0,1.0,563.0,3,0.0,0.0,0.0,0.0,0.0
4,1,7,1,9E,N904XJ,ATL,SGF,950,947.0,-3.0,0.0,0900-0959,25.0,4.0,1053,1044.0,-9.0,0.0,1000-1059,0.0,123.0,1.0,563.0,3,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7422032,12,31,2,B6,N193JB,MCO,SWF,1356,1500.0,64.0,1.0,1300-1359,20.0,5.0,1639,1731.0,52.0,1.0,1600-1659,0.0,163.0,1.0,989.0,4,52.0,0.0,0.0,0.0,0.0
7422033,12,31,2,B6,N304JB,DCA,BOS,1420,1414.0,-6.0,0.0,1400-1459,15.0,7.0,1550,1533.0,-17.0,0.0,1500-1559,0.0,90.0,1.0,399.0,2,0.0,0.0,0.0,0.0,0.0
7422034,12,31,2,B6,N193JB,PHL,BOS,700,652.0,-8.0,0.0,0700-0759,12.0,5.0,825,751.0,-34.0,0.0,0800-0859,0.0,85.0,1.0,280.0,2,0.0,0.0,0.0,0.0,0.0
7422035,12,31,2,B6,N563JB,BOS,SJU,813,812.0,-1.0,0.0,0800-0859,10.0,3.0,1315,1248.0,-27.0,0.0,1300-1359,0.0,242.0,1.0,1674.0,7,0.0,0.0,0.0,0.0,0.0


In [9]:
def val_freq(col='', df=df):
    i = 0
    for v in df[col].value_counts().sort_index():
        print("{} : {} records ({:.2f}%)" \
              .format(df[col].value_counts().sort_index().index[i], v,  v / len(df) * 100))
        i += 1

In [10]:
%%time

for col in df.columns:
    print(col, ':', df[col].nunique(), 'unique values')
    if df[col].nunique() < 50:
        val_freq(col)
    print("")

MONTH : 12 unique values
1 : 583985 records (7.87%)
2 : 533175 records (7.18%)
3 : 632074 records (8.52%)
4 : 612023 records (8.25%)
5 : 636390 records (8.57%)
6 : 636691 records (8.58%)
7 : 659029 records (8.88%)
8 : 658461 records (8.87%)
9 : 605979 records (8.16%)
10 : 636014 records (8.57%)
11 : 602453 records (8.12%)
12 : 625763 records (8.43%)

DAY_OF_MONTH : 31 unique values
1 : 240951 records (3.25%)
2 : 237619 records (3.20%)
3 : 242174 records (3.26%)
4 : 241829 records (3.26%)
5 : 238517 records (3.21%)
6 : 244304 records (3.29%)
7 : 243985 records (3.29%)
8 : 248962 records (3.35%)
9 : 240393 records (3.24%)
10 : 247271 records (3.33%)
11 : 249266 records (3.36%)
12 : 243253 records (3.28%)
13 : 245085 records (3.30%)
14 : 244813 records (3.30%)
15 : 249956 records (3.37%)
16 : 240931 records (3.25%)
17 : 246247 records (3.32%)
18 : 250135 records (3.37%)
19 : 244596 records (3.30%)
20 : 243522 records (3.28%)
21 : 247257 records (3.33%)
22 : 250716 records (3.38%)
23 : 242

In [11]:
# Since 'FLIGHTS' value is always 1, it does not add any information. let's just drop it:
df.drop('FLIGHTS', axis=1, inplace=True)

In [12]:
df.shape

(7422037, 28)

In [13]:
%memit

peak memory: 1877.73 MiB, increment: 0.00 MiB


In [14]:
# Focus on delayed flights:
delays = df[df['ARR_DEL15'] == 1]
delays

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,DEP_TIME_BLK,TAXI_OUT,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,ARR_TIME_BLK,CANCELLED,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,3,4,9E,N195PQ,TYS,ATL,1140,1205.0,25.0,1.0,1100-1159,30.0,4.0,1250,1315.0,25.0,1.0,1200-1259,0.0,70.0,152.0,1,0.0,0.0,0.0,0.0,25.0
1,1,4,5,9E,N919XJ,TYS,ATL,1140,1250.0,70.0,1.0,1100-1159,35.0,9.0,1250,1412.0,82.0,1.0,1200-1259,0.0,70.0,152.0,1,0.0,0.0,12.0,0.0,70.0
8,1,11,5,9E,N135EV,ATL,SGF,950,945.0,-5.0,0.0,0900-0959,33.0,24.0,1053,1113.0,20.0,1.0,1000-1059,0.0,123.0,563.0,3,0.0,0.0,20.0,0.0,0.0
12,1,15,2,9E,N917XJ,ATL,SGF,950,1043.0,53.0,1.0,0900-0959,14.0,5.0,1053,1136.0,43.0,1.0,1000-1059,0.0,123.0,563.0,3,11.0,0.0,0.0,0.0,32.0
16,1,19,6,9E,N937XJ,ATL,SGF,950,1001.0,11.0,0.0,0900-0959,40.0,5.0,1051,1113.0,22.0,1.0,1000-1059,0.0,121.0,563.0,3,0.0,0.0,11.0,0.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7421989,12,31,2,B6,N523JB,MCO,SJU,1428,1732.0,184.0,1.0,1400-1459,24.0,7.0,1815,2126.0,191.0,1.0,1800-1859,0.0,167.0,1189.0,5,184.0,0.0,7.0,0.0,0.0
7422004,12,31,2,B6,N198JB,BOS,RDU,1545,1619.0,34.0,1.0,1500-1559,16.0,6.0,1759,1821.0,22.0,1.0,1700-1759,0.0,134.0,612.0,3,8.0,0.0,0.0,0.0,14.0
7422009,12,31,2,B6,N603JB,SLC,MCO,2210,2231.0,21.0,1.0,2200-2259,29.0,5.0,417,438.0,21.0,1.0,0001-0559,0.0,247.0,1931.0,8,7.0,0.0,0.0,0.0,14.0
7422019,12,31,2,B6,N317JB,MCO,DCA,1853,1844.0,-9.0,0.0,1800-1859,12.0,68.0,2100,2139.0,39.0,1.0,2100-2159,0.0,127.0,759.0,4,0.0,0.0,39.0,0.0,0.0


In [15]:
# Let's check now the main reasons behind delays:
causes = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
delay_groups = delays.groupby('ARR_DEL15')[causes].agg(['count', 'median', 'mean', 'sum', 'max'])
# delay_groups.columns = ['_'.join(col).strip() for col in delay_groups.columns.values]
delay_groups.T.loc[:, 1].apply(lambda x: '%.0f' % x)

CARRIER_DELAY        count      1389253
                     median           0
                     mean            21
                     sum       29352960
                     max           2695
WEATHER_DELAY        count      1389253
                     median           0
                     mean             4
                     sum        5282501
                     max           1847
NAS_DELAY            count      1389253
                     median           2
                     mean            17
                     sum       23044854
                     max           1741
SECURITY_DELAY       count      1389253
                     median           0
                     mean             0
                     sum         133484
                     max           1078
LATE_AIRCRAFT_DELAY  count      1389253
                     median           3
                     mean            27
                     sum       38075922
                     max           2206


There are some missing values. Let's further delve into it:

In [16]:
%%time

# Absolute & Relative frequency of missing values by column:
pd.set_option('display.max_rows', df.shape[1])
missing = pd.DataFrame([df.isna().sum(), df.isna().sum() / len(df) * 100], index=['Absolute', 'Relative']).T.sort_values(by='Relative', ascending=False)
missing

Wall time: 5.04 s


,Absolute,Relative
ARR_DEL15,153805.0,2.072275
ARR_DELAY,153805.0,2.072275
TAXI_IN,137647.0,1.854572
ARR_TIME,137646.0,1.854558
TAXI_OUT,133977.0,1.805124
DEP_DEL15,130110.0,1.753023
DEP_DELAY,130110.0,1.753023
DEP_TIME,130086.0,1.752699
TAIL_NUM,17837.0,0.240325
CRS_ELAPSED_TIME,135.0,0.001819


In [17]:
# Show which columns present more than 5% of missing data:
empty_df = missing[missing['Relative'] > 5]
empty_df

,Absolute,Relative


In [18]:
%memit

peak memory: 2217.91 MiB, increment: 0.00 MiB


In [19]:
isolated_elements_missing = missing.loc[(missing['Relative'] < 5) & (missing['Relative'] > 0)]
isolated_elements_missing

,Absolute,Relative
ARR_DEL15,153805.0,2.072275
ARR_DELAY,153805.0,2.072275
TAXI_IN,137647.0,1.854572
ARR_TIME,137646.0,1.854558
TAXI_OUT,133977.0,1.805124
DEP_DEL15,130110.0,1.753023
DEP_DELAY,130110.0,1.753023
DEP_TIME,130086.0,1.752699
TAIL_NUM,17837.0,0.240325
CRS_ELAPSED_TIME,135.0,0.001819


In [20]:
%%time

# Quick approach → check how many rows contain empty values to see if directly dropping them would be feasible:

# Check which rows have at least 1 NaN:
df[df.isna().any(axis=1)]

Wall time: 2.42 s


,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,DEP_TIME_BLK,TAXI_OUT,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,ARR_TIME_BLK,CANCELLED,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
26,1,29,2,9E,N931XJ,ATL,SGF,950,NaN,NaN,NaN,0900-0959,NaN,NaN,1053,NaN,NaN,NaN,1000-1059,1.0,123.0,563.0,3,0.0,0.0,0.0,0.0,0.0
53,1,29,2,9E,N931XJ,SGF,ATL,1128,NaN,NaN,NaN,1100-1159,NaN,NaN,1420,NaN,NaN,NaN,1400-1459,1.0,112.0,563.0,3,0.0,0.0,0.0,0.0,0.0
142,1,20,7,9E,N934XJ,LGA,BTV,1420,NaN,NaN,NaN,1400-1459,NaN,NaN,1545,NaN,NaN,NaN,1500-1559,1.0,85.0,258.0,2,0.0,0.0,0.0,0.0,0.0
298,1,24,4,9E,N930XJ,LGA,BTV,2159,NaN,NaN,NaN,2100-2159,NaN,NaN,2321,NaN,NaN,NaN,2300-2359,1.0,82.0,258.0,2,0.0,0.0,0.0,0.0,0.0
406,1,22,2,9E,N929XJ,DTW,BTV,2020,NaN,NaN,NaN,2000-2059,NaN,NaN,2218,NaN,NaN,NaN,2200-2259,1.0,118.0,537.0,3,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7419670,12,18,3,B6,N348JB,EWR,BOS,2104,NaN,NaN,NaN,2100-2159,NaN,NaN,2216,NaN,NaN,NaN,2200-2259,1.0,72.0,200.0,1,0.0,0.0,0.0,0.0,0.0
7419672,12,18,3,B6,N329JB,RDU,BOS,530,NaN,NaN,NaN,0001-0559,NaN,NaN,723,NaN,NaN,NaN,0700-0759,1.0,113.0,612.0,3,0.0,0.0,0.0,0.0,0.0
7419848,12,19,4,B6,N306JB,SYR,BOS,530,NaN,NaN,NaN,0001-0559,NaN,NaN,651,NaN,NaN,NaN,0600-0659,1.0,81.0,265.0,2,0.0,0.0,0.0,0.0,0.0
7420968,12,30,1,B6,N579JB,BOS,PBI,1936,2057.0,81.0,1.0,1900-1959,53.0,NaN,2303,NaN,NaN,NaN,2300-2359,0.0,207.0,1197.0,5,0.0,0.0,0.0,0.0,0.0


In [21]:
%%time

# DF's length as is:
original_length = len(df)
print('Original dataset length:\t', original_length)

# Check how many rows have at least 1 NaN:
manipulated_length = len(df.drop(df[df.isna().any(axis=1)].index, axis=0))
print('Manipulated dataset length:\t', manipulated_length)

# Dropped rows, absolute and relative number:
print('{} rows deleted, accounting for {:.2f}% of the original dataset.'.format(original_length - manipulated_length, (original_length - manipulated_length) / original_length * 100))

Original dataset length:	 7422037
Manipulated dataset length:	 7268232
153805 rows deleted, accounting for 2.07% of the original dataset.
Wall time: 4.01 s


Considering that it is only ~2% of the data, and that the dataset is big enough, let's just simply drop those rows as a quick  cleaning method.

In [22]:
%%time

df = df.drop(df[df.isna().any(axis=1)].index, axis=0)
df

Wall time: 3.63 s


,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,DEP_TIME_BLK,TAXI_OUT,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,ARR_TIME_BLK,CANCELLED,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,3,4,9E,N195PQ,TYS,ATL,1140,1205.0,25.0,1.0,1100-1159,30.0,4.0,1250,1315.0,25.0,1.0,1200-1259,0.0,70.0,152.0,1,0.0,0.0,0.0,0.0,25.0
1,1,4,5,9E,N919XJ,TYS,ATL,1140,1250.0,70.0,1.0,1100-1159,35.0,9.0,1250,1412.0,82.0,1.0,1200-1259,0.0,70.0,152.0,1,0.0,0.0,12.0,0.0,70.0
2,1,5,6,9E,N316PQ,ATL,SGF,950,956.0,6.0,0.0,0900-0959,20.0,3.0,1051,1043.0,-8.0,0.0,1000-1059,0.0,121.0,563.0,3,0.0,0.0,0.0,0.0,0.0
3,1,6,7,9E,N325PQ,ATL,SGF,950,945.0,-5.0,0.0,0900-0959,16.0,3.0,1053,1029.0,-24.0,0.0,1000-1059,0.0,123.0,563.0,3,0.0,0.0,0.0,0.0,0.0
4,1,7,1,9E,N904XJ,ATL,SGF,950,947.0,-3.0,0.0,0900-0959,25.0,4.0,1053,1044.0,-9.0,0.0,1000-1059,0.0,123.0,563.0,3,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7422032,12,31,2,B6,N193JB,MCO,SWF,1356,1500.0,64.0,1.0,1300-1359,20.0,5.0,1639,1731.0,52.0,1.0,1600-1659,0.0,163.0,989.0,4,52.0,0.0,0.0,0.0,0.0
7422033,12,31,2,B6,N304JB,DCA,BOS,1420,1414.0,-6.0,0.0,1400-1459,15.0,7.0,1550,1533.0,-17.0,0.0,1500-1559,0.0,90.0,399.0,2,0.0,0.0,0.0,0.0,0.0
7422034,12,31,2,B6,N193JB,PHL,BOS,700,652.0,-8.0,0.0,0700-0759,12.0,5.0,825,751.0,-34.0,0.0,0800-0859,0.0,85.0,280.0,2,0.0,0.0,0.0,0.0,0.0
7422035,12,31,2,B6,N563JB,BOS,SJU,813,812.0,-1.0,0.0,0800-0859,10.0,3.0,1315,1248.0,-27.0,0.0,1300-1359,0.0,242.0,1674.0,7,0.0,0.0,0.0,0.0,0.0


In [23]:
%%time

# CHECK AGAIN after manipulating the missing data.
# Absolute & Relative frequency of missing values by column:
pd.set_option('display.max_rows', 110) # It greatly slows down the output display and freezes the kernel
missing_2 = pd.DataFrame([df.isna().sum(), df.isna().sum() / len(df) * 100], index=['Absolute', 'Relative']).T.sort_values(by='Relative', ascending=False)
missing_2

Wall time: 4.92 s


,Absolute,Relative
MONTH,0.0,0.0
DAY_OF_MONTH,0.0,0.0
SECURITY_DELAY,0.0,0.0
NAS_DELAY,0.0,0.0
WEATHER_DELAY,0.0,0.0
CARRIER_DELAY,0.0,0.0
DISTANCE_GROUP,0.0,0.0
DISTANCE,0.0,0.0
CRS_ELAPSED_TIME,0.0,0.0
CANCELLED,0.0,0.0


Great. Now the dataset is totally free of missing values.

In [24]:
# 'TAXI_OUT' time is not an input, since there is no way to predict its actual value in advance.
# However, there is indeed some prior knowledge about it, based on past data concerning to operations on each airport.
# Each carrier may optimize its airport operations in a different manner, thus leading to average differences between them.
# Therefore, let's see whether some value could be used as a baseline for each pair and hence use it as an input feature.

DepDel_TaxOutTim = df.groupby(['ORIGIN', 'OP_UNIQUE_CARRIER'])[['TAXI_OUT', 'DEP_DELAY']] \
                     .agg({'TAXI_OUT' : ['count', 'mean', 'median', 'min', 'max'],
                           'DEP_DELAY' : ['mean', 'median', 'min', 'max']})
DepDel_TaxOutTim

TAXI_OUT                                DEP_DELAY  \
                            count       mean median  min    max       mean   
ORIGIN OP_UNIQUE_CARRIER                                                     
ABE    9E                     626  16.915335   14.0  6.0  114.0   7.865815   
       DL                     326  18.377301   14.0  7.0  112.0   1.638037   
       EV                     137  18.518248   15.0  5.0   82.0  27.401460   
       G4                    1142  10.722417   10.0  5.0   83.0  10.619965   
       MQ                     509  14.290766   12.0  5.0   85.0  11.618861   
...                           ...        ...    ...  ...    ...        ...   
XNA    YX                    1751  18.055397   15.0  4.0  141.0  14.347230   
XWA    OO                     204  26.769608   19.0  7.0   95.0  31.995098   
YAK    AS                     694  10.207493    9.0  2.0   45.0  -6.237752   
YUM    OO                    1556  14.491645   13.0  2.0   87.0   3.267995   
       YV                     353  13.070822   12.0  4.0   36.0  10.467422   

                                               
                         median   min     max  
ORIGIN OP_UNIQUE_CARRIER                       
ABE    9E                  -3.0 -18.0   360.0  
       DL                  -3.0 -13.0   150.0  
       EV                  -4.0 -19.0   382.0  
       G4                  -5.0 -32.0  1855.0  
       MQ                  -6.0 -14.0   739.0  
...                         ...   ...     ...  
XNA    YX                  -4.0 -22.0   505.0  
XWA    OO                   0.0 -21.0   897.0  
YAK    AS                 -12.0 -56.0   212.0  
YUM    OO                  -6.0 -24.0   640.0  
       YV                  -3.0 -18.0   526.0  

[1977 rows x 9 columns]

In [25]:
# As a means of checking whether the median could be taken as an input, let's first verify how much variance there is for
# each particular airport/carrier pair, as well as how far are the extreme values.

Dep_relative_diff = DepDel_TaxOutTim.loc[:, ('TAXI_OUT', 'mean')] / DepDel_TaxOutTim.loc[:, ('TAXI_OUT', 'median')]
Dep_relative_diff.describe()

# Based on the results, it is observed that the median is commonly close to the mean for each pair.
# In cases where this assumption is not satisfied, it is normally due to outliers.
# These extreme values significantly move the means to the right, but are not representative in general.

# In a nutshell, it is fair to assume that each airport/carrier median is a ground baseline value for an input feature.

count    1977.000000
mean        1.169532
std         0.106970
min         0.877193
25%         1.108320
50%         1.157459
75%         1.218919
max         3.977778
dtype: float64

In [26]:
%%time
# As the solution in the cell above (in 'Raw NBConvert' format) takes so long, an incredibly faster approach is addressed:
aux = df.groupby(['ORIGIN', 'OP_UNIQUE_CARRIER'], as_index=False)[['TAXI_OUT']].median()
df = df.reset_index().merge(aux, how='inner', on=['ORIGIN', 'OP_UNIQUE_CARRIER'], suffixes=(None,'_median')).set_index('index').sort_index()
df

Wall time: 20.3 s


,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,DEP_TIME_BLK,TAXI_OUT,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,ARR_TIME_BLK,CANCELLED,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,TAXI_OUT_median
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,3,4,9E,N195PQ,TYS,ATL,1140,1205.0,25.0,1.0,1100-1159,30.0,4.0,1250,1315.0,25.0,1.0,1200-1259,0.0,70.0,152.0,1,0.0,0.0,0.0,0.0,25.0,15.0
1,1,4,5,9E,N919XJ,TYS,ATL,1140,1250.0,70.0,1.0,1100-1159,35.0,9.0,1250,1412.0,82.0,1.0,1200-1259,0.0,70.0,152.0,1,0.0,0.0,12.0,0.0,70.0,15.0
2,1,5,6,9E,N316PQ,ATL,SGF,950,956.0,6.0,0.0,0900-0959,20.0,3.0,1051,1043.0,-8.0,0.0,1000-1059,0.0,121.0,563.0,3,0.0,0.0,0.0,0.0,0.0,17.0
3,1,6,7,9E,N325PQ,ATL,SGF,950,945.0,-5.0,0.0,0900-0959,16.0,3.0,1053,1029.0,-24.0,0.0,1000-1059,0.0,123.0,563.0,3,0.0,0.0,0.0,0.0,0.0,17.0
4,1,7,1,9E,N904XJ,ATL,SGF,950,947.0,-3.0,0.0,0900-0959,25.0,4.0,1053,1044.0,-9.0,0.0,1000-1059,0.0,123.0,563.0,3,0.0,0.0,0.0,0.0,0.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7422032,12,31,2,B6,N193JB,MCO,SWF,1356,1500.0,64.0,1.0,1300-1359,20.0,5.0,1639,1731.0,52.0,1.0,1600-1659,0.0,163.0,989.0,4,52.0,0.0,0.0,0.0,0.0,15.0
7422033,12,31,2,B6,N304JB,DCA,BOS,1420,1414.0,-6.0,0.0,1400-1459,15.0,7.0,1550,1533.0,-17.0,0.0,1500-1559,0.0,90.0,399.0,2,0.0,0.0,0.0,0.0,0.0,15.0
7422034,12,31,2,B6,N193JB,PHL,BOS,700,652.0,-8.0,0.0,0700-0759,12.0,5.0,825,751.0,-34.0,0.0,0800-0859,0.0,85.0,280.0,2,0.0,0.0,0.0,0.0,0.0,17.0


In [27]:
# Similarly to what happens with 'TAXI_OUT', 'TAXI_IN' time is not an input, (unpredictable actual value in advance).
# However, the Arrival times dynamic follow the same commonalities as assumed for Departure.
# Therefore, let's proceed likewise and check the results.

TaxInTim_ArrDel = df.groupby(['DEST', 'OP_UNIQUE_CARRIER'])[['TAXI_IN', 'ARR_DELAY']] \
                     .agg({'TAXI_IN' : ['count', 'mean', 'median', 'min', 'max'],
                           'ARR_DELAY' : ['mean', 'median', 'min', 'max']})
TaxInTim_ArrDel

TAXI_IN                              ARR_DELAY         \
                         count      mean median  min   max       mean median   
DEST OP_UNIQUE_CARRIER                                                         
ABE  9E                    629  5.076312    4.0  2.0  28.0  -0.233704  -10.0   
     DL                    326  4.542945    4.0  2.0  14.0  -1.368098  -11.5   
     EV                    137  4.832117    5.0  2.0  17.0  38.649635    7.0   
     G4                   1145  5.524017    5.0  4.0  75.0   1.863755   -7.0   
     MQ                    512  4.285156    4.0  1.0  40.0   6.503906   -7.0   
...                        ...       ...    ...  ...   ...        ...    ...   
XNA  YX                   1767  7.233729    6.0  2.0  88.0   3.688738   -7.0   
XWA  OO                    202  6.331683    5.0  1.0  36.0  -2.995050  -15.0   
YAK  AS                    697  4.173601    4.0  2.0  33.0  -0.256815   -6.0   
YUM  OO                   1553  4.046362    3.0  2.0  27.0   4.652930   -5.0   
     YV                    353  3.929178    4.0  1.0  16.0  11.549575   -1.0   

                                      
                         min     max  
DEST OP_UNIQUE_CARRIER                
ABE  9E                -37.0   461.0  
     DL                -34.0   465.0  
     EV                -29.0   355.0  
     G4                -37.0   555.0  
     MQ                -37.0   597.0  
...                      ...     ...  
XNA  YX                -57.0   736.0  
XWA  OO                -47.0   369.0  
YAK  AS                -47.0   232.0  
YUM  OO                -36.0  1218.0  
     YV                -22.0   825.0  

[1976 rows x 9 columns]

In [28]:
# Similarly to what has been done for Departure:

Arr_relative_diff = TaxInTim_ArrDel.loc[:, ('TAXI_IN', 'mean')] / TaxInTim_ArrDel.loc[:, ('TAXI_IN', 'median')]
Arr_relative_diff.describe()

# Once again, it is observed that the median is commonly close to the mean for each pair.

# Therefore, it is fair to assume that each airport/carrier median is a ground baseline value for an input feature.

count    1976.000000
mean        1.152699
std         0.149455
min         0.838818
25%         1.057349
50%         1.128030
75%         1.216900
max         2.904865
dtype: float64

In [29]:
%%time
# As the solution in the cell above (in 'Raw NBConvert' format) takes so long, an incredibly faster approach is addressed:
aux = df.groupby(['DEST', 'OP_UNIQUE_CARRIER'], as_index=False)[['TAXI_IN']].median()
df = df.reset_index().merge(aux, how='inner', on=['DEST', 'OP_UNIQUE_CARRIER'], suffixes=(None,'_median')).set_index('index').sort_index()
df

Wall time: 26 s


,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,DEP_TIME_BLK,TAXI_OUT,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,ARR_TIME_BLK,CANCELLED,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,TAXI_OUT_median,TAXI_IN_median
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,3,4,9E,N195PQ,TYS,ATL,1140,1205.0,25.0,1.0,1100-1159,30.0,4.0,1250,1315.0,25.0,1.0,1200-1259,0.0,70.0,152.0,1,0.0,0.0,0.0,0.0,25.0,15.0,8.0
1,1,4,5,9E,N919XJ,TYS,ATL,1140,1250.0,70.0,1.0,1100-1159,35.0,9.0,1250,1412.0,82.0,1.0,1200-1259,0.0,70.0,152.0,1,0.0,0.0,12.0,0.0,70.0,15.0,8.0
2,1,5,6,9E,N316PQ,ATL,SGF,950,956.0,6.0,0.0,0900-0959,20.0,3.0,1051,1043.0,-8.0,0.0,1000-1059,0.0,121.0,563.0,3,0.0,0.0,0.0,0.0,0.0,17.0,5.0
3,1,6,7,9E,N325PQ,ATL,SGF,950,945.0,-5.0,0.0,0900-0959,16.0,3.0,1053,1029.0,-24.0,0.0,1000-1059,0.0,123.0,563.0,3,0.0,0.0,0.0,0.0,0.0,17.0,5.0
4,1,7,1,9E,N904XJ,ATL,SGF,950,947.0,-3.0,0.0,0900-0959,25.0,4.0,1053,1044.0,-9.0,0.0,1000-1059,0.0,123.0,563.0,3,0.0,0.0,0.0,0.0,0.0,17.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7422032,12,31,2,B6,N193JB,MCO,SWF,1356,1500.0,64.0,1.0,1300-1359,20.0,5.0,1639,1731.0,52.0,1.0,1600-1659,0.0,163.0,989.0,4,52.0,0.0,0.0,0.0,0.0,15.0,6.0
7422033,12,31,2,B6,N304JB,DCA,BOS,1420,1414.0,-6.0,0.0,1400-1459,15.0,7.0,1550,1533.0,-17.0,0.0,1500-1559,0.0,90.0,399.0,2,0.0,0.0,0.0,0.0,0.0,15.0,6.0
7422034,12,31,2,B6,N193JB,PHL,BOS,700,652.0,-8.0,0.0,0700-0759,12.0,5.0,825,751.0,-34.0,0.0,0800-0859,0.0,85.0,280.0,2,0.0,0.0,0.0,0.0,0.0,17.0,6.0


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7268232 entries, 0 to 7422036
Data columns (total 30 columns):
 #   Column               Dtype  
---  ------               -----  
 0   MONTH                int64  
 1   DAY_OF_MONTH         int64  
 2   DAY_OF_WEEK          int64  
 3   OP_UNIQUE_CARRIER    object 
 4   TAIL_NUM             object 
 5   ORIGIN               object 
 6   DEST                 object 
 7   CRS_DEP_TIME         int64  
 8   DEP_TIME             float64
 9   DEP_DELAY            float64
 10  DEP_DEL15            float64
 11  DEP_TIME_BLK         object 
 12  TAXI_OUT             float64
 13  TAXI_IN              float64
 14  CRS_ARR_TIME         int64  
 15  ARR_TIME             float64
 16  ARR_DELAY            float64
 17  ARR_DEL15            float64
 18  ARR_TIME_BLK         object 
 19  CANCELLED            float64
 20  CRS_ELAPSED_TIME     float64
 21  DISTANCE             float64
 22  DISTANCE_GROUP       int64  
 23  CARRIER_DELAY        float64
 24

In [31]:
# Create new categorical features based on CRS departure/arrival times.
df['DEP_TIME_BLK'] = df['DEP_TIME_BLK'].rename('DEP_TIME_hour').str.slice(0, 2)
df['ARR_TIME_BLK'] = df['ARR_TIME_BLK'].rename('ARR_TIME_hour').str.slice(0, 2)
df = df.rename({'DEP_TIME_BLK' : 'DEP_TIME_hour', 'ARR_TIME_BLK' : 'ARR_TIME_hour'}, axis=1)

In [36]:
# Casting numeric types to 'int64':
float64_cols = df.select_dtypes('float64').columns
df[float64_cols[:-2]] = df[float64_cols[:-2]].astype('int64')
# Casting certain features (numeric and string) types to 'category':
ord_cat_cols = ['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DEP_TIME_hour', 'ARR_TIME_hour']
notord_cat_cols = ['OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DEP_DEL15', 'ARR_DEL15', 'CANCELLED']
df[ord_cat_cols] = df[ord_cat_cols].astype('category')
df[notord_cat_cols] = df[notord_cat_cols].astype('category')

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7268232 entries, 0 to 7422036
Data columns (total 30 columns):
 #   Column               Dtype   
---  ------               -----   
 0   MONTH                category
 1   DAY_OF_MONTH         category
 2   DAY_OF_WEEK          category
 3   OP_UNIQUE_CARRIER    category
 4   TAIL_NUM             object  
 5   ORIGIN               category
 6   DEST                 category
 7   CRS_DEP_TIME         int64   
 8   DEP_TIME             int64   
 9   DEP_DELAY            int64   
 10  DEP_DEL15            category
 11  DEP_TIME_hour        category
 12  TAXI_OUT             int64   
 13  TAXI_IN              int64   
 14  CRS_ARR_TIME         int64   
 15  ARR_TIME             int64   
 16  ARR_DELAY            int64   
 17  ARR_DEL15            category
 18  ARR_TIME_hour        category
 19  CANCELLED            category
 20  CRS_ELAPSED_TIME     int64   
 21  DISTANCE             int64   
 22  DISTANCE_GROUP       int64   
 23  CARRIER

Only by casting features to their corresponding dtypes, there is a memory usage reduction of 500MB (~ 30%).

#### Run only the first time to generate the global CLEAN file (year 2019)

In [42]:
%%time

output_csv_path = os.path.join(root,
                               "Output_Data",
                               "US_DoT",
                               "AL_OTP_MVP_Preprocessed_19_v2_clean.csv")

df.to_csv(path_or_buf=output_csv_path,
          index=False,
          encoding='latin1')
          
# Wall time: 5min 31s

Wall time: 1min 14s


In [43]:
t1 = time.perf_counter() - t0
print("Time elapsed: ", t1) # CPU seconds elapsed (floating point)

Time elapsed:  953.3264606


___

___